# Deep Learning Final Assignment  – Marc Glowienke

To properly run this notebook, the data has to be downloaded from https://drive.google.com/file/d/1G_Exgw9WXI6swzGQEyzueOQvB_01mUJt/view to be in the correct form and folder structure

In [ ]:
## Preamble
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import h5py
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle
from scipy.signal import decimate
import os
from os import listdir
from os.path import isfile, join

tf.random.set_seed(123)
np.random.seed(123)


def get_dataset_name(file_name_with_dir): # Function used to load files
    filename_without_dir = file_name_with_dir.split('/')[-1]
    temp = filename_without_dir.split('_')[:-1]
    dataset_name = "_".join(temp)
    return dataset_name

def load_intra():
    ## Train data
    x = []
    y = []
    paths = ["Final_Project_DL/Intra/train/rest_105923_%.0f.h5","Final_Project_DL/Intra/train/task_motor_105923_%.0f.h5",
             "Final_Project_DL/Intra/train/task_story_math_105923_%.0f.h5",
             "Final_Project_DL/Intra/train/task_working_memory_105923_%.0f.h5"]
    for path in paths:
        for k in range(1,9):
            filename_path= path %k
            with h5py.File(filename_path,'r') as f:
                dataset_name = get_dataset_name(filename_path)
                matrix = f.get(dataset_name)[()]
                x.append(matrix) #create x frame

                # transfrom class to number code
                if dataset_name.split("_")[-2] == 'rest':
                    Class = 0
                elif dataset_name.split("_")[-2] == 'motor':
                    Class = 1
                elif dataset_name.split("_")[-2] == 'math':
                    Class = 2
                elif dataset_name.split("_")[-2] == 'memory':
                    Class = 3
                else: print('Error in adding classes')

                y.append([int(dataset_name.split("_")[-1]),Class])
    intra_train_x = np.array(x)
    intra_train_y = np.array(y)
    del x,y
    ## Test data
    x = []
    y = []
    paths = ["Final_Project_DL/Intra/test/rest_105923_%.0f.h5","Final_Project_DL/Intra/test/task_motor_105923_%.0f.h5",
             "Final_Project_DL/Intra/test/task_story_math_105923_%.0f.h5",
             "Final_Project_DL/Intra/test/task_working_memory_105923_%.0f.h5"]
    for path in paths:
        for k in range(9,11):
            filename_path= path %k
            with h5py.File(filename_path,'r') as f:
                dataset_name = get_dataset_name(filename_path)
                matrix = f.get(dataset_name)[()]
                x.append(matrix) #create x frame

                # transfrom class to number code
                if dataset_name.split("_")[-2] == 'rest':
                    Class = 0
                elif dataset_name.split("_")[-2] == 'motor':
                    Class = 1
                elif dataset_name.split("_")[-2] == 'math':
                    Class = 2
                elif dataset_name.split("_")[-2] == 'memory':
                    Class = 3
                else: print('Error in adding classes')

                y.append([int(dataset_name.split("_")[-1]),Class])
    intra_test_x = np.array(x)
    intra_test_y = np.array(y)
    del x,y

    print("Intra: ",intra_train_x.shape, intra_train_y.shape, intra_test_x.shape, intra_test_y.shape)
    return intra_train_x, intra_train_y, intra_test_x, intra_test_y

def path_list(file_path):
    train_paths = []
    d = file_path
    for path in os.listdir(d):
        full_path = os.path.join(d, path)
        if os.path.isfile(full_path):
            train_paths.append(full_path)
    return train_paths


def concatenate_data(data_path):
    data = []
    labels = []

    for path in data_path:
        with h5py.File(path,'r') as f:
            dataset_name = get_dataset_name(path)
            matrix = f.get(dataset_name)[()]
            data.append(matrix)

            if dataset_name.split("_")[-2] == 'rest':
                Class = 0
            elif dataset_name.split("_")[-2] == 'motor':
                Class = 1
            elif dataset_name.split("_")[-2] == 'math':
                Class = 2
            elif dataset_name.split("_")[-2] == 'memory':
                Class = 3

            labels.append([int(dataset_name.split("_")[-1]),Class])
    return np.array(data), np.array(labels)

def load_cross():
    train_path = path_list('Final_Project_DL/Cross/train/')
    train_data, train_data_labels = concatenate_data(train_path)

    old_test1_path = sorted(path_list('Final_Project_DL/Cross/test1/'))
    test1_order = [0, 2, 3, 1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
    test1_path = [old_test1_path[i] for i in test1_order]
    test1_data, test1_data_labels = concatenate_data(test1_path)

    old_test2_path = sorted(path_list('Final_Project_DL/Cross/test2/'))
    test2_order = [0, 2, 3, 1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
    test2_path = [old_test2_path[i] for i in test2_order]
    test2_data, test2_data_labels = concatenate_data(test2_path)

    old_test3_path = sorted(path_list('Final_Project_DL/Cross/test3/'))
    test3_order = [0, 2, 3, 1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
    test3_path = [old_test3_path[i] for i in test3_order]
    test3_data, test3_data_labels = concatenate_data(test3_path)

    print(train_data.shape,train_data_labels.shape,test1_data.shape,test1_data_labels.shape,
          test2_data.shape,test2_data_labels.shape,test3_data.shape,test3_data_labels.shape)
    return train_data,train_data_labels,test1_data,test1_data_labels,test2_data,test2_data_labels,test3_data,test3_data_labels

def data_wrangle(train_x,train_y,DOWNSAMPLE_RATE):
    train_x = decimate(train_x,DOWNSAMPLE_RATE) # decrease the sampling rate

    train_labels = tf.keras.utils.to_categorical(train_y[:,1]) # make labels to 1-hot encoding

    ## Scaling per chunk to [0,1], so all channels are scaled using the same min and max per chunk
    train_x_scaled = np.empty(train_x.shape)
    for chunk in range(train_x.shape[0]):
        maxval = np.amax(train_x[chunk,:,:])
        minval = np.amin(train_x[chunk,:,:])
        train_x_scaled[chunk,:,:] = ((train_x[chunk,:,:] - minval)/(maxval - minval))

    train_x, train_labels = shuffle(train_x_scaled,train_labels,random_state = 0) # shuffle samples and labels
    train_x = np.swapaxes(train_x,1,2) # change to shape, where features are last


    print("Wrangle: ",train_x.shape, train_labels.shape)
    return train_x, train_labels

def data_windows(x,y, window_size): #Split data into windows of certain length,
    data = np.empty((1,WINDOW_SIZE,248)) # Ommit end part to have equal length
    labels = np.empty((1,4))

    for chunk in range(x.shape[0]):
        for k in range(WINDOW_SIZE,int(np.floor(x.shape[1]/WINDOW_SIZE)*WINDOW_SIZE)+1,WINDOW_SIZE):
            indices = range(k - WINDOW_SIZE,k)
            sample = x[chunk,indices,:]
            data = np.append(data,sample.reshape(1,WINDOW_SIZE,x.shape[2]),axis = 0)
            labels = np.append(labels,y[chunk].reshape((1,4)),axis=0)
    print("WINDOW: ",data[1:,:,:].shape, labels[1:,:].shape)

    X, Y = shuffle(data[1:,:,:], labels[1:,:],random_state = 0)
    return X, Y

#### MODEL
def train_lstm(train_x,train_labels,epochs,batch_size):
    mc = tf.keras.callbacks.ModelCheckpoint('best_model_lstm.h5', monitor='val_accuracy', mode='max',verbose = 1, save_best_only=True)
    model = tf.keras.models.Sequential([
        #tf.keras.layers.BatchNormalization(),
        tf.keras.layers.LSTM(200,input_shape=train_x.shape[-2:]),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(100, activation='relu'),
        tf.keras.layers.Dense(100, activation='relu'),
        tf.keras.layers.Dense(train_labels.shape[1], activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    history = model.fit(train_x, train_labels, epochs=epochs, batch_size=batch_size, 
                        validation_split = 0.2,callbacks = [mc],verbose = 0)
    model = tf.keras.models.load_model('best_model_lstm.h5')

    print("---------------------------------------- \n")
    print(model.summary())

    loss = history.history['loss']
    val_loss = history.history['val_loss']
    accuracy_plot = history.history['accuracy']
    plt.figure(figsize= (20,10))
    plt.plot(range(len(loss)), loss, 'b', label='Training loss')
    plt.plot(range(len(loss)), accuracy_plot, 'r', label='Accuracy')
    plt.plot(range(len(loss)), val_loss, 'orange', label='Validation loss')
    plt.legend()
    plt.title("Loss for LSTM Model")
    plt.savefig('loss_lstm.pdf',bbox_inches = 'tight')
    plt.show()

    return model

def evaluate_lstm(model,test_x,test_labels,batch_size):
    _, accuracy = model.evaluate(test_x, test_labels, batch_size=batch_size, verbose=0)
    print("---------------------------------------- \n")

    predictions = tf.argmax(model.predict(test_x), axis = 1)
    labels = tf.argmax(test_labels, axis = 1)
    confusion_matrix = tf.math.confusion_matrix(
        labels, predictions, num_classes=None, weights=None, dtype=tf.dtypes.int32, name=None)

    return accuracy, confusion_matrix


def train_cnn(train_x,train_labels,epochs,batch_size,filters,kernels):
    # define model
    mc = tf.keras.callbacks.ModelCheckpoint('best_model_cnn.h5', monitor='val_accuracy', mode='max',verbose = 1, save_best_only=True)
    model = tf.keras.models.Sequential([
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv1D(filters=filters, kernel_size=kernels, activation='relu', input_shape=train_x.shape[-2:]),
    tf.keras.layers.Conv1D(filters=filters, kernel_size=kernels, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(train_labels.shape[1], activation='softmax'),
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    history = model.fit(train_x, train_labels, epochs=epochs, batch_size=batch_size, 
                        validation_split = 0.2,callbacks = [mc],verbose = 0)
    
    model = tf.keras.models.load_model('best_model_cnn.h5')

    print("---------------------------------------- \n")
    print(model.summary())

    loss = history.history['loss']
    val_loss = history.history['val_loss']
    accuracy_plot = history.history['accuracy']
    plt.figure(figsize= (20,10))
    plt.plot(range(len(loss)), loss, 'b', label='Training loss')
    plt.plot(range(len(loss)), accuracy_plot, 'r', label='Accuracy')
    plt.plot(range(len(loss)), val_loss, 'orange', label='Validation loss')
    plt.legend()
    plt.title("Loss for CNN Model")
    plt.savefig('loss_cnn.pdf',bbox_inches = 'tight')
    plt.show()

    return model


def evaluate_cnn(model,test_x,test_labels,batch_size):
    # evaluate model
    _, accuracy = model.evaluate(test_x, test_labels, batch_size=batch_size, verbose=0)

    print("---------------------------------------- \n")

    predictions = tf.argmax(model.predict(test_x), axis = 1)
    labels = tf.argmax(test_labels, axis = 1)
    confusion_matrix = tf.math.confusion_matrix(
        labels, predictions, num_classes=None, weights=None, dtype=tf.dtypes.int32, name=None)

    return accuracy,confusion_matrix

accuracy_lstm1 = None
accuracy_cnn1 = None
accuracy_cnn = None
accuracy_lstm = None

In [ ]:
##### RUN
### HYPERPARAMETER COLLECTION
DOWNSAMPLE_RATE = 10
EPOCHS_LSTM = 50
EPOCHS_CNN = 100
BATCH_SIZE = 68
WINDOW_SIZE = 200

In [ ]:
intra_train_x, intra_train_y, intra_test_x, intra_test_y = load_intra()

train_x, train_labels = data_wrangle(intra_train_x, intra_train_y,DOWNSAMPLE_RATE = DOWNSAMPLE_RATE)
test_x, test_labels = data_wrangle(intra_test_x, intra_test_y,DOWNSAMPLE_RATE = DOWNSAMPLE_RATE)
intra_train_x, intra_train_labels = data_windows(train_x,train_labels,window_size = WINDOW_SIZE)
intra_test_x, intra_test_labels = data_windows(test_x, test_labels,window_size = WINDOW_SIZE)
del intra_train_y,intra_test_y

In [ ]:
cross_train_x, cross_train_y, cross_test1_x, cross_test1_y, cross_test2_x, cross_test2_y, cross_test3_x, cross_test3_y = load_cross()
train_x, train_labels = data_wrangle(cross_train_x, cross_train_y,DOWNSAMPLE_RATE = DOWNSAMPLE_RATE)
train_x, train_labels = data_windows(train_x,train_labels,window_size = WINDOW_SIZE)
del cross_train_x, cross_train_y

In [ ]:
test_x, test_labels = data_wrangle(cross_test1_x, cross_test1_y,DOWNSAMPLE_RATE = DOWNSAMPLE_RATE)
test1_x, test1_labels = data_windows(test_x, test_labels,window_size = WINDOW_SIZE)

test_x, test_labels = data_wrangle(cross_test2_x, cross_test2_y,DOWNSAMPLE_RATE = DOWNSAMPLE_RATE)
test2_x, test2_labels = data_windows(test_x, test_labels,window_size = WINDOW_SIZE)

test_x, test_labels = data_wrangle(cross_test3_x, cross_test3_y,DOWNSAMPLE_RATE = DOWNSAMPLE_RATE)
test3_x, test3_labels = data_windows(test_x,test_labels,window_size = WINDOW_SIZE)
del test_x, test_labels

In [ ]:
print("-------------------------------------------------------------------------------- \n")
model_lstm = train_lstm(intra_train_x,intra_train_labels,epochs = EPOCHS_LSTM,batch_size = BATCH_SIZE)

accuracy_lstm,confusion_matrix_lstm = evaluate_lstm(model_lstm,intra_test_x,intra_test_labels,batch_size = BATCH_SIZE)
model_cnn = train_cnn(intra_train_x,intra_train_labels,epochs = EPOCHS_CNN,batch_size = BATCH_SIZE,filters = 64,kernels = 3)
accuracy_cnn,confusion_matrix_cnn = evaluate_cnn(model_cnn,intra_test_x,intra_test_labels,batch_size = BATCH_SIZE)
print("---------------------------------------- \n")
if accuracy_lstm != None:
    print("RESULTS LSTM: \n")
    print("epochs: %.0f" %EPOCHS_LSTM)
    print("Accuracy on the test set for LSTM %.4f \n" %accuracy_lstm)
    print("\n Confusion Matrix LSTM:")
    print(confusion_matrix_lstm)
if accuracy_cnn != None:
    print("RESULTS CNN: \n")
    print("epochs: %.0f \n" %EPOCHS_CNN)
    print("Accuracy on the test set for CNN %.4f" %accuracy_cnn)
    print("\n Confusion Matrix CNN:")
    print(confusion_matrix_cnn)

In [ ]:
model_lstm = train_lstm(train_x,train_labels,epochs = EPOCHS_LSTM,batch_size = BATCH_SIZE)
model_cnn = train_cnn(train_x,train_labels,epochs = EPOCHS_CNN,batch_size = BATCH_SIZE, filters = 64,kernels = 3)

accuracy_lstm1,confusion_matrix_lstm1 = evaluate_lstm(model_lstm,test1_x,test1_labels,batch_size = BATCH_SIZE)
accuracy_cnn1,confusion_matrix_cnn1 = evaluate_cnn(model_cnn,test1_x,test1_labels,batch_size = BATCH_SIZE)

accuracy_lstm2,confusion_matrix_lstm2 = evaluate_lstm(model_lstm,test2_x,test2_labels,batch_size = BATCH_SIZE)
accuracy_cnn2,confusion_matrix_cnn2 = evaluate_cnn(model_cnn,test2_x,test2_labels,batch_size = BATCH_SIZE)

accuracy_lstm3,confusion_matrix_lstm3 = evaluate_lstm(model_lstm,test3_x,test3_labels,batch_size = BATCH_SIZE)
accuracy_cnn3,confusion_matrix_cnn3 = evaluate_cnn(model_cnn,test3_x,test3_labels,batch_size = BATCH_SIZE)

print("---------------------------------------- \n")
if accuracy_lstm1 != None:
    print("RESULTS LSTM: \n")
    print("epochs: %.0f" %EPOCHS_LSTM)
    print("Accuracy on test set 1 for LSTM %.4f \n" %accuracy_lstm1)
    print("Accuracy on test set 2 for LSTM %.4f \n" %accuracy_lstm2)
    print("Accuracy on test set 3 for LSTM %.4f \n" %accuracy_lstm3)
    print("\n Confusion Matrix LSTM 1:")
    print(confusion_matrix_lstm1)
    print("\n Confusion Matrix LSTM 2:")
    print(confusion_matrix_lstm2)
    print("\n Confusion Matrix LSTM 3:")
    print(confusion_matrix_lstm3)
if accuracy_cnn1 != None:
    print("RESULTS CNN: \n")
    print("epochs: %.0f \n" %EPOCHS_CNN)
    print("Accuracy on test set 1 for CNN %.4f \n" %accuracy_cnn1)
    print("Accuracy on test set 2 for CNN %.4f \n" %accuracy_cnn2)
    print("Accuracy on test set 3 for CNN %.4f \n" %accuracy_cnn3)
    print("\n Confusion Matrix CNN 1:")
    print(confusion_matrix_cnn1)
    print("\n Confusion Matrix CNN 2:")
    print(confusion_matrix_cnn2)
    print("\n Confusion Matrix CNN 3:")
    print(confusion_matrix_cnn3)